In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
#import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# Write out mutation counts
mutations = pandas.read_csv("../data/mutations.csv.bz2", converters={'binding_peptides': eval})
mutations = mutations.ix[mutations.source_id.str.startswith("AOCS")]
mutations

/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,source_id,donor,called,genome,contig,interbase_start,interbase_end,ref,alt,effect,gene,context_5_prime,context_3_prime,context_mutation,dna_alt_reads,dna_ref_reads,dna_total_reads,rna_alt_reads,rna_ref_reads,rna_total_reads,vaf,any_alt_reads,snv,mutation_id,site_id,context_A,context_C,context_T,context_G,context_GC,unique_to_treated,context_mutation_5p,context_mutation_3p,context_mutation_3p5p,sources,binding_affinity,binding_allele,num_alt,num_ref,total_depth,binder,binding_peptides,num_binders
0,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,61407797,61407798,C,T,intergenic,None,CCCCTCATCTCCCAG,CTGAACCACGTCCTT,C>T,0,47,47,NaN,NaN,NaN,0.000000,False,True,18:61407797 C>T,18:61407797,0.166667,0.500000,0.233333,0.100000,0.600000,True,G(C>T),(C>T)C,G(C>T)C,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
1,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,61407797,61407798,C,T,intergenic,None,CCCCTCATCTCCCAG,CTGAACCACGTCCTT,C>T,5,19,24,NaN,NaN,NaN,0.208332,True,True,18:61407797 C>T,18:61407797,0.166667,0.500000,0.233333,0.100000,0.600000,True,G(C>T),(C>T)C,G(C>T)C,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
2,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,6,63262672,63262673,C,A,intergenic,None,GACCACACAGGAACC,GAGAATTTACATGCA,C>A,0,69,69,NaN,NaN,NaN,0.000000,False,True,6:63262672 C>A,6:63262672,0.400000,0.266667,0.133333,0.200000,0.466667,True,C(C>A),(C>A)G,C(C>A)G,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
3,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,6,63262672,63262673,C,A,intergenic,None,GACCACACAGGAACC,GAGAATTTACATGCA,C>A,9,40,49,NaN,NaN,NaN,0.183673,True,True,6:63262672 C>A,6:63262672,0.400000,0.266667,0.133333,0.200000,0.466667,True,C(C>A),(C>A)G,C(C>A)G,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
4,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,63487173,63487174,C,G,intronic,CDH7,TGTTCCTTTGCTTCC,TAAAGTGCGAAAGAT,C>G,0,31,31,NaN,NaN,NaN,0.000000,False,True,18:63487173 C>G,18:63487173,0.233333,0.200000,0.366667,0.200000,0.400000,True,C(C>G),(C>G)T,C(C>G)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
5,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,63487173,63487174,C,G,intronic,CDH7,TGTTCCTTTGCTTCC,TAAAGTGCGAAAGAT,C>G,6,17,23,NaN,NaN,NaN,0.260868,True,True,18:63487173 C>G,18:63487173,0.233333,0.200000,0.366667,0.200000,0.400000,True,C(C>G),(C>G)T,C(C>G)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
6,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,68110672,68110673,A,T,intergenic,None,TCCTTGTCGTCTCGT,TCTAGCCTGCAAACC,T>A,5,27,32,NaN,NaN,NaN,0.156250,True,True,18:68110672 A>T,18:68110672,0.133333,0.366667,0.333333,0.166667,0.533333,True,T(T>A),(T>A)T,T(T>A)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
7,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,68110672,68110673,A,T,intergenic,None,TCCTTGTCGTCTCGT,TCTAGCCTGCAAACC,T>A,11,33,44,NaN,NaN,NaN,0.249999,True,True,18:68110672 A>T,18:68110672,0.133333,0.366667,0.333333,0.166667,0.533333,True,T(T>A),(T>A)T,T(T>A)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
8,AOCS-139-16/SP101896,AOCS-139,1,GRCh37,18,68385066,68385067,C,G,intergenic,None,TCCAGACAGACCACA,AGATTCACAGCCGAA,C>G,12,27,39,NaN,NaN,NaN,0.307692,True,True,18:68385066 C>G,18:68385066,0.400000,0.333333,0.100000,0.166667,0.500000,True,A(C>G),(C>G)A,A(C>G)A,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
9,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,68385066,68385067,C,G,intergenic,None,TCCAGACAGACCACA,AGATTCACAGCCGAA,C>G,9,43,52,NaN,NaN,NaN,0.173077,True,True,18:68385066 C>G,18:68385066,0.400000,0.333333,0.100000,0.166667,0.500000,True,A(C>G),(C>G)A,A(C>G)A,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0


In [3]:
sources = pandas.read_csv("../data/sources.csv", index_col="source_id")
sources["source_id"] = sources.index
sources = sources.ix[sources.cohort == "AOCS"]
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

,donor,cohort,library,treated,timepoint,metastasis,tissue_type,interval_days,assay,paired,source_id,treated_paired
source_id,,,,,,,,,,,,
AOCS-001-1/SP101515,AOCS-001,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-001-1/SP101515,False
AOCS-002-1/SP101517,AOCS-002,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-002-1/SP101517,False
AOCS-004-1/SP101519,AOCS-004,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-004-1/SP101519,False
AOCS-005-1/SP101521,AOCS-005,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-005-1/SP101521,False
AOCS-034-13/SP101524,AOCS-034,AOCS,WGS,True,recurrence,False,ascites,1597,WGS,True,AOCS-034-13/SP101524,True
AOCS-034-1/SP101523,AOCS-034,AOCS,WGS,False,primary,False,solid,0,WGS,True,AOCS-034-1/SP101523,True
AOCS-055-1/SP101526,AOCS-055,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-055-1/SP101526,False
AOCS-056-1/SP101528,AOCS-056,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-056-1/SP101528,False
AOCS-057-1/SP101530,AOCS-057,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-057-1/SP101530,False


In [4]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-034-13/SP101524,71,86,7,66,72,44,17,86,80,33,58,74,43,33,22,65,76,25,37,42,23,5,39,36,89,73,11,136,44,39,14,84,72,287,36,268,48,48,60,126,39,47,44,61,19,16,38,30,68,84,5,47,46,49,5,73,86,79,43,69,14,19,22,34,36,32,25,41,12,12,23,19,102,80,10,103,333,82,18,497,266,100,41,188,39,34,30,58,50,39,16,40,32,18,38,70,new
AOCS-064-13/SP101554,146,158,11,194,127,61,32,140,175,82,65,136,83,77,82,129,108,69,74,145,50,23,66,69,122,110,13,131,80,59,24,82,83,83,58,194,144,109,137,230,58,119,89,259,35,46,77,194,155,159,9,104,64,157,6,118,126,148,44,112,54,56,40,106,52,92,42,85,18,49,50,80,85,103,10,130,100,95,18,186,88,68,36,125,87,82,44,189,96,112,37,129,51,39,70,162,new
AOCS-065-13/SP101560,28,23,3,30,24,15,5,43,33,18,24,26,17,11,15,32,26,10,12,28,9,4,14,9,29,24,3,28,25,12,5,28,31,44,21,48,19,23,13,28,7,14,16,22,7,2,14,24,28,14,1,23,9,18,1,29,32,30,24,28,10,14,15,11,14,12,1,18,11,4,12,12,30,30,2,26,33,17,2,38,26,31,16,26,7,19,11,21,21,10,6,25,18,7,13,20,new
AOCS-086-13/SP101606,101,127,6,87,83,42,23,93,81,57,87,96,44,30,43,61,86,34,65,93,32,11,37,29,104,83,11,151,64,50,21,91,127,364,49,321,125,112,149,197,60,71,69,120,25,20,49,53,119,131,13,102,47,118,9,120,121,111,65,113,17,30,24,40,47,53,41,48,16,13,31,25,77,74,8,106,122,76,11,206,110,84,34,109,72,44,27,79,60,61,32,73,34,14,48,78,new
AOCS-088-13/SP101612,204,236,17,209,134,79,27,125,190,99,94,201,81,67,84,144,123,85,74,143,43,19,74,49,185,181,22,211,89,75,19,120,124,138,66,236,113,150,140,238,66,138,64,87,24,33,75,75,133,120,15,150,61,71,11,93,142,90,79,134,62,75,69,142,56,51,44,65,35,16,66,53,216,166,21,269,327,156,33,514,415,192,48,325,100,102,62,208,91,100,47,99,46,39,95,89,new
AOCS-091-13/SP101624,141,176,15,133,169,63,23,121,160,77,124,129,63,69,54,97,137,72,116,125,31,18,55,37,156,136,13,188,85,44,12,129,85,165,93,198,141,125,188,228,67,87,77,113,33,30,64,51,102,153,17,100,64,93,9,132,160,110,93,114,47,47,48,82,65,76,54,90,22,22,59,40,113,107,10,132,122,92,11,213,108,86,51,108,100,78,51,141,100,85,51,100,52,31,56,82,new
AOCS-092-13/SP101630,80,164,2,110,45,18,6,44,53,26,55,58,42,64,30,45,42,30,33,49,23,10,33,34,149,82,24,257,27,24,10,68,152,547,47,528,347,184,302,383,71,52,58,117,23,21,48,33,83,163,7,100,20,112,4,65,112,108,48,64,23,39,19,30,24,72,19,27,8,25,19,20,69,79,10,132,54,34,8,139,84,95,37,130,45,32,29,48,28,35,17,41,10,16,33,42,new
AOCS-093-13/SP101638,60,83,4,64,48,24,13,40,49,34,29,57,29,15,39,43,65,34,43,53,13,8,21,20,43,53,6,77,31,47,15,57,56,127,27,126,85,61,109,86,35,44,49,57,12,15,35,25,56,72,8,45,23,65,7,52,72,56,24,51,15,18,29,22,26,37,21,29,7,9,15,10,58,39,7,66,89,61,6,130,103,53,18,75,33,19,19,50,42,35,22,47,24,18,26,42,new
AOCS-094-2/SP106641,98,107,9,102,96,61,26,105,117,53,65,127,45,42,45,87,94,59,72,103,35,11,42,24,101,95,6,86,59,54,18,94,57,41,49,108,49,68,63,101,45,110,55,64,15,29,58,28,47,59,9,67,40,33,9,52,69,48,45,65,28,27,25,53,48,47,41,47,22,7,39,32,98,56,9,97,170,86,19,227,90,60,28,101,70,41,40,88,80,109,46,112,38,26,56,73,new


In [5]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


0


context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1/SP101515,126,94,15,93,84,48,12,83,95,53,123,107,30,35,44,60,125,48,81,120,24,15,32,32,112,94,14,84,55,40,12,73,75,79,74,110,45,62,73,69,36,70,50,45,17,16,26,31,66,73,9,56,44,42,5,41,93,78,83,68,29,20,43,56,56,29,48,68,4,5,36,27,81,106,19,104,149,90,12,235,158,111,64,129,51,52,28,91,67,48,38,81,23,21,35,64,all
AOCS-002-1/SP101517,37,23,6,24,17,8,5,11,30,21,68,30,18,18,13,19,49,12,35,51,13,3,8,9,27,16,1,16,7,8,5,10,26,34,56,41,6,7,10,11,9,9,9,16,2,5,4,10,11,4,4,14,7,8,0,4,18,23,46,28,4,6,9,9,15,11,6,13,2,3,4,8,23,25,5,29,14,6,2,23,26,33,24,44,12,8,8,29,17,5,7,20,6,6,8,16,all
AOCS-004-1/SP101519,111,112,17,87,78,47,12,72,138,68,131,101,56,59,64,83,155,60,105,155,23,15,34,33,138,83,13,134,58,52,26,79,104,78,104,134,39,44,57,79,87,81,88,118,22,31,46,94,50,50,10,57,39,36,8,48,94,86,123,109,39,34,39,48,93,62,75,110,17,12,33,39,72,85,10,120,59,70,15,147,144,103,57,125,80,46,42,104,112,94,41,96,27,30,62,69,all
AOCS-005-1/SP101521,107,67,6,62,56,38,18,53,123,55,167,101,50,28,33,72,125,58,100,127,32,17,37,53,58,48,10,46,43,32,14,56,86,60,143,86,24,38,32,54,43,51,46,74,22,18,31,60,68,43,10,54,25,19,10,31,77,68,134,63,24,13,13,31,50,25,33,50,25,10,44,35,85,86,21,121,148,75,12,252,209,105,91,179,62,15,19,68,58,50,37,80,36,20,44,84,all
AOCS-034-1/SP101523,169,110,16,96,116,57,16,107,163,76,144,106,57,41,51,97,188,66,75,113,44,16,30,33,99,104,16,102,67,56,22,106,101,80,93,116,47,67,69,72,50,55,52,106,19,32,52,48,72,69,11,75,50,44,10,78,111,72,106,86,36,30,44,54,64,30,67,51,34,32,279,46,101,134,16,121,169,95,12,252,170,119,74,149,69,42,33,105,97,79,35,95,44,21,44,93,all
AOCS-034-13/SP101524,240,207,27,163,189,98,31,186,241,111,195,171,94,70,74,152,263,85,106,151,67,21,65,68,192,182,27,254,111,96,34,195,180,420,133,425,96,111,131,205,94,102,101,165,38,48,92,73,144,184,17,123,96,104,17,161,194,157,158,161,51,50,69,91,91,63,90,86,43,43,302,64,216,224,26,231,515,183,28,788,472,218,124,338,108,73,62,160,138,110,48,134,75,39,79,163,all
AOCS-055-1/SP101526,155,125,17,138,149,79,42,149,185,96,318,144,55,52,65,117,247,112,169,286,43,8,57,64,135,93,14,121,127,91,38,148,130,116,183,206,52,61,77,101,104,140,97,156,39,49,82,122,91,80,19,70,65,59,15,110,143,99,222,109,41,39,46,77,117,74,89,140,34,25,71,62,265,162,27,203,1617,388,98,1657,1044,347,227,609,94,54,51,130,137,122,71,146,88,39,68,153,all
AOCS-056-1/SP101528,181,163,15,130,131,59,16,108,144,78,187,138,71,63,98,90,126,64,96,117,21,13,53,35,119,130,13,102,78,50,19,86,72,86,122,150,47,81,78,104,58,74,39,57,23,31,58,46,90,99,16,78,38,33,11,77,111,88,125,95,38,40,43,80,51,27,42,60,22,7,52,29,101,134,9,105,78,74,8,167,108,86,90,114,68,61,38,124,60,60,39,86,25,18,48,80,all
AOCS-057-1/SP101530,123,128,6,99,105,34,10,96,134,66,134,93,63,33,57,77,136,55,84,116,32,10,29,29,102,114,17,109,49,37,9,80,82,70,99,99,44,81,59,74,57,76,52,88,14,22,47,41,68,51,19,61,30,36,7,56,102,81,100,76,31,31,42,65,52,41,52,53,18,10,38,25,95,86,12,125,86,72,14,155,93,108,66,88,73,39,45,108,82,70,39,93,28,18,52,68,all


In [6]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/mutation_contexts_counts.csv", index=False)
muts_r

,source_id,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,A[C>T]G,A[C>T]T,A[T>A]A,A[T>A]C,A[T>A]G,A[T>A]T,A[T>C]A,A[T>C]C,A[T>C]G,A[T>C]T,A[T>G]A,A[T>G]C,A[T>G]G,A[T>G]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,C[C>G]A,C[C>G]C,C[C>G]G,C[C>G]T,C[C>T]A,C[C>T]C,C[C>T]G,C[C>T]T,C[T>A]A,C[T>A]C,C[T>A]G,C[T>A]T,C[T>C]A,C[T>C]C,C[T>C]G,C[T>C]T,C[T>G]A,C[T>G]C,C[T>G]G,C[T>G]T,G[C>A]A,G[C>A]C,G[C>A]G,G[C>A]T,G[C>G]A,G[C>G]C,G[C>G]G,G[C>G]T,G[C>T]A,G[C>T]C,G[C>T]G,G[C>T]T,G[T>A]A,G[T>A]C,G[T>A]G,G[T>A]T,G[T>C]A,G[T>C]C,G[T>C]G,G[T>C]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[C>A]A,T[C>A]C,T[C>A]G,T[C>A]T,T[C>G]A,T[C>G]C,T[C>G]G,T[C>G]T,T[C>T]A,T[C>T]C,T[C>T]G,T[C>T]T,T[T>A]A,T[T>A]C,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1/SP101515,AOCS-001-1/SP101515,126,94,15,93,84,48,12,83,95,53,123,107,30,35,44,60,125,48,81,120,24,15,32,32,112,94,14,84,55,40,12,73,75,79,74,110,45,62,73,69,36,70,50,45,17,16,26,31,66,73,9,56,44,42,5,41,93,78,83,68,29,20,43,56,56,29,48,68,4,5,36,27,81,106,19,104,149,90,12,235,158,111,64,129,51,52,28,91,67,48,38,81,23,21,35,64,all
AOCS-002-1/SP101517,AOCS-002-1/SP101517,37,23,6,24,17,8,5,11,30,21,68,30,18,18,13,19,49,12,35,51,13,3,8,9,27,16,1,16,7,8,5,10,26,34,56,41,6,7,10,11,9,9,9,16,2,5,4,10,11,4,4,14,7,8,0,4,18,23,46,28,4,6,9,9,15,11,6,13,2,3,4,8,23,25,5,29,14,6,2,23,26,33,24,44,12,8,8,29,17,5,7,20,6,6,8,16,all
AOCS-004-1/SP101519,AOCS-004-1/SP101519,111,112,17,87,78,47,12,72,138,68,131,101,56,59,64,83,155,60,105,155,23,15,34,33,138,83,13,134,58,52,26,79,104,78,104,134,39,44,57,79,87,81,88,118,22,31,46,94,50,50,10,57,39,36,8,48,94,86,123,109,39,34,39,48,93,62,75,110,17,12,33,39,72,85,10,120,59,70,15,147,144,103,57,125,80,46,42,104,112,94,41,96,27,30,62,69,all
AOCS-005-1/SP101521,AOCS-005-1/SP101521,107,67,6,62,56,38,18,53,123,55,167,101,50,28,33,72,125,58,100,127,32,17,37,53,58,48,10,46,43,32,14,56,86,60,143,86,24,38,32,54,43,51,46,74,22,18,31,60,68,43,10,54,25,19,10,31,77,68,134,63,24,13,13,31,50,25,33,50,25,10,44,35,85,86,21,121,148,75,12,252,209,105,91,179,62,15,19,68,58,50,37,80,36,20,44,84,all
AOCS-034-1/SP101523,AOCS-034-1/SP101523,169,110,16,96,116,57,16,107,163,76,144,106,57,41,51,97,188,66,75,113,44,16,30,33,99,104,16,102,67,56,22,106,101,80,93,116,47,67,69,72,50,55,52,106,19,32,52,48,72,69,11,75,50,44,10,78,111,72,106,86,36,30,44,54,64,30,67,51,34,32,279,46,101,134,16,121,169,95,12,252,170,119,74,149,69,42,33,105,97,79,35,95,44,21,44,93,all
AOCS-034-13/SP101524,AOCS-034-13/SP101524,240,207,27,163,189,98,31,186,241,111,195,171,94,70,74,152,263,85,106,151,67,21,65,68,192,182,27,254,111,96,34,195,180,420,133,425,96,111,131,205,94,102,101,165,38,48,92,73,144,184,17,123,96,104,17,161,194,157,158,161,51,50,69,91,91,63,90,86,43,43,302,64,216,224,26,231,515,183,28,788,472,218,124,338,108,73,62,160,138,110,48,134,75,39,79,163,all
AOCS-055-1/SP101526,AOCS-055-1/SP101526,155,125,17,138,149,79,42,149,185,96,318,144,55,52,65,117,247,112,169,286,43,8,57,64,135,93,14,121,127,91,38,148,130,116,183,206,52,61,77,101,104,140,97,156,39,49,82,122,91,80,19,70,65,59,15,110,143,99,222,109,41,39,46,77,117,74,89,140,34,25,71,62,265,162,27,203,1617,388,98,1657,1044,347,227,609,94,54,51,130,137,122,71,146,88,39,68,153,all
AOCS-056-1/SP101528,AOCS-056-1/SP101528,181,163,15,130,131,59,16,108,144,78,187,138,71,63,98,90,126,64,96,117,21,13,53,35,119,130,13,102,78,50,19,86,72,86,122,150,47,81,78,104,58,74,39,57,23,31,58,46,90,99,16,78,38,33,11,77,111,88,125,95,38,40,43,80,51,27,42,60,22,7,52,29,101,134,9,105,78,74,8,167,108,86,90,114,68,61,38,124,60,60,39,86,25,18,48,80,all
AOCS-057-1/SP101530,AOCS-057-1/SP101530,123,128,6,99,105,34,10,96,134,66,134,93,63,33,57,77,136,55,84,116,32,10,29,29,102,114,17,109,49,37,9,80,82,70,99,99,44,81,59,74,57,76,52,88,14,22,47,41,68,51,19,61,30,36,7,56,102,81,100,76,31,31,42,65,52,41,52,53,18,10,38,25,95,86,12,125,86,72,14,155,93,